#### Utilisation de base :

In [ ]:
# READRHEOLOGY : reads

import rheol_functions as rh

# Load file
file = 'Z:/AntonPaar/Carbopol 20230906_full.csv'

data = rh.read_rheology(file)                       # Read the file
data = rh.assign_steps(data, [3,4], 'flowcurve')    # Say that steps 3 and 4 are `flow curves`
rh.list_steps(data)                                 # List all the steps
flowcurves = rh.slice(data,[3])                     # Select step 3
fig, fit = rh.plot_flowcurve(flowcurves)                       # Plot the flow curves of step 3

In [ ]:
# Your plots here !

#### Utilisation pour LAOS

In [ ]:
# Example of a LAOS raw data plot for Anton Paar
# The Anton Paar LAOS Step (n°7 here) has multiple oscillation
# amplitudes. 

import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt


rawlaos = data[data['raw'] & (data['step'] == 7)]
synlaos = data[~data['raw'] & (data['step'] == 7)]

cmap = cm.magma_r

%matplotlib widget

fig = plt.figure()
ax0 = fig.add_axes([0.10,0.55,0.4,0.45])
ax1 = fig.add_axes([0.10,0.10,0.4,0.45])
ax2 = fig.add_axes([0.58,0.10,0.4,0.45])
ax3 = fig.add_axes([0.58,0.55,0.4,0.45])

for pt in np.unique(rawlaos['point'][:]):
    color = 0.85*np.array(cmap(pt/8))
    select = rawlaos[rawlaos['point'] == pt]
    f = np.squeeze(np.unique(select['freq']))
    t_rebuilt = np.linspace(0, 1/f, len(select))*2*np.pi*f

    proj_gam = rh.proj_fourier(t_rebuilt, select['raw_oscstrain'], nmodes=10)
    proj_tau = rh.proj_fourier(t_rebuilt, select['raw_oscstress'], nmodes=10)
    fit_gam = rh.build_fourier(proj_gam, t_rebuilt, nmodes=10)
    fit_tau = rh.build_fourier(proj_tau, t_rebuilt, nmodes=10)

    # Plotting
    ax0.plot(t_rebuilt, select['raw_oscstrain'], '-',  color=color)
    ax0.plot(t_rebuilt, fit_gam, ':',  color=color)
    ax1.plot(t_rebuilt, select['raw_oscstress'], '-', color=color)
    ax1.plot(t_rebuilt, fit_tau, ':', color=color)
    ax2.semilogy(proj_tau['mode'][1::2], proj_tau['amp'][1::2], ':o', color=color)
    ax3.semilogy(proj_gam['mode'][1::2], proj_gam['amp'][1::2], ':o', color=color)
    ax2.semilogy(proj_tau['mode'][::2], proj_tau['amp'][::2], 'o', markeredgecolor=color, markerfacecolor='None')
    ax3.semilogy(proj_gam['mode'][::2], proj_gam['amp'][::2], 'o', markeredgecolor=color, markerfacecolor='None') 

# Format plot
ax1.set_xlabel('$\\omega t$')
ax1.set_ylabel('$\\tau$ (Pa)')
ax0.set_ylabel('$\\gamma$ (1)')
ax2.set_xlabel('Mode')
ax2.set_ylabel('Amp ($\\tau$)')
ax3.set_ylabel('Amp ($\\gamma$)')
ax2.set_ylim([1e-3*np.max(proj_tau['mode']), 5*np.max(proj_tau['mode'])])
ax3.set_ylim([1e-3*np.max(proj_tau['mode']), 5*np.max(proj_tau['mode'])])
fig.set_figwidth(12)